Developed by **Chang Ye** @ 09-13-2025

Feel free to contect me via email: chang.ye@my.cityu.edu.hk | and obtain the latest version from github: https://github.com/Yechang618/Quant-Analysis-example.git


---
What's new?

This is an updated version of backtest engine (BTEngine). In this version, the backtest for one of the most recent Generate AI model, Kronos, is included. For more information about Kronos, please refer to https://github.com/shiyu-coder/Kronos/tree/master. 
Thanks for this mavelous work of Yu Shi's group.


**Section 0** Install the python libraries in case they are not installed by default. Please comment this block if you don't need it.

In [2]:
# Basic libraries
!pip install yfinance pandas matplotlib torch
!pip install einops safetensors hf_xet

**Section 1** Library loading

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
### system related libraries
import sys, os
# print(os.getcwd()) # check current address.
## Import the backtest toolbox (BTT)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
from BTToolbox import BTEngine as BTE
from BTToolbox import utils as uts

**Section 2** Download the lastest version of Kronos from github.

In [4]:
!git clone https://github.com/shiyu-coder/Kronos.git

fatal: destination path 'Kronos' already exists and is not an empty directory.


In [5]:
Kronos_path = os.path.join(os.getcwd(),'Kronos')
if Kronos_path not in sys.path:
    sys.path.append(Kronos_path)


**Section 3**: Data download and preprocess

In [6]:
"""
Main function to demonstrate backtesting with real data
"""
### Initialize backtest engine
backtester = BTE.BacktestEngine(initial_capital=10000, commission=0.001, slippage=0.0005)

### Fetch real-world data (Apple stock)
# Here we select AAPL as the stock
# You may test other stock, such as tickers in https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
TICKER = "AAPL"
START_DATE = "2020-01-01"
END_DATE = "2023-12-31"
LOOKBACK = 300 # The window size for machine learning approaches
## 
bTdata = uts.backTestData(ticker = TICKER, start_date = START_DATE, end_date = END_DATE, lookback = LOOKBACK)
## Get targeted data
dft, ts = bTdata.df_target.copy(), bTdata.ts_target.copy()
dft.index = ts
backtester.data = dft


Fetching data for AAPL from 2020-01-01 to 2023-12-31...


[*********************100%***********************]  1 of 1 completed

Data fetched: 1006 records from 2020-01-02 00:00:00 to 2023-12-29 00:00:00
Preprocessed data with 301 lookbacks and 706 targets...


In [7]:
### Test different strategies
strategies = ['kronos', 'trend_following', 'mean_reversion', 'macd_crossover', 'rsi_momentum', 'breakout']

df_pred_kronos = bTdata.generate_kronos_predictions()

results = {}
for strategy in strategies:
    print(f"\n{'='*60}")
    print(f"Testing {strategy.upper()} strategy")
    print(f"{'='*60}")

    # Run backtest
    if strategy == 'kronos':
        # The startegy for Kronos is based on its prediction on close,
        # so we need to input the prediced dataframe, i.e. df_pred_kronos
        equity, trades = backtester.run_backtest(strategy=strategy, df_pred_kronos = df_pred_kronos)
    else:
        equity, trades = backtester.run_backtest(strategy=strategy)

    # Calculate performance metrics
    metrics = backtester.calculate_performance_metrics(benchmark_ticker='^GSPC')
    results[strategy] = metrics

    # Print results
    print("Final Capital: ", metrics['Final Capital'])
    print("Total Return: ", metrics['Total Return (%)'])
    print("Benchmark Return: ", metrics['Benchmark Return (%)'])
    print("Alpha: ", metrics['Alpha (%)'])
    print("Number of Trades: ", metrics['Number of Trades'])
    print("Win Rate: ", metrics['Win Rate (%)'])

    # Plot results for the first strategy only to avoid too many plots
    # if strategy == strategies[0]:
    #     backtester.plot_results()



### Compare strategies
print(f"\n{'='*80}")
print("STRATEGY COMPARISON")
print(f"{'='*80}")

comparison_df = pd.DataFrame(results).T
important_metrics = ['Total Return (%)', 'Benchmark Return (%)', 'Alpha (%)',
                    'Sharpe Ratio', 'Max Drawdown (%)', 'Win Rate (%)', 'Number of Trades']

print(comparison_df[important_metrics].round(2))

Generating Kronos predictions from 2021-03-12 00:00:00  to 2023-12-29 00:00:00
Predictions generated as: 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 706 entries, 2021-03-12 to 2023-12-29
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   open        706 non-null    float64       
 1   high        706 non-null    float64       
 2   low         706 non-null    float64       
 3   close       706 non-null    float64       
 4   volume      706 non-null    int64         
 5   amount      706 non-null    float64       
 6   timestamps  706 non-null    datetime64[ns]
 7   close_pred  706 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 49.6 KB
None

Testing KRONOS strategy
Running backtest with kronos strategy...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Final Capital:  9524.75145501749
Total Return:  -4.752485449825105
Benchmark Return:  Ticker
^GSPC    21.301992
dtype: float64
Alpha:  Ticker
^GSPC   -26.054478
dtype: float64
Number of Trades:  147
Win Rate:  44.89795918367347

Testing TREND_FOLLOWING strategy
Running backtest with trend_following strategy...
Final Capital:  9745.103872143765
Total Return:  -2.548961278562345
Benchmark Return:  Ticker
^GSPC    21.301992
dtype: float64
Alpha:  Ticker
^GSPC   -23.850954
dtype: float64
Number of Trades:  7
Win Rate:  14.285714285714285

Testing MEAN_REVERSION strategy
Running backtest with mean_reversion strategy...



[*********************100%***********************]  1 of 1 completed


Final Capital:  11646.7513369227
Total Return:  16.467513369227007
Benchmark Return:  Ticker
^GSPC    21.301992
dtype: float64
Alpha:  Ticker
^GSPC   -4.834479
dtype: float64
Number of Trades:  2
Win Rate:  50.0

Testing MACD_CROSSOVER strategy
Running backtest with macd_crossover strategy...
Final Capital:  11474.337728256154
Total Return:  14.743377282561543
Benchmark Return:  Ticker
^GSPC    21.301992
dtype: float64
Alpha:  Ticker
^GSPC   -6.558615
dtype: float64
Number of Trades:  11
Win Rate:  36.36363636363637

Testing RSI_MOMENTUM strategy
Running backtest with rsi_momentum strategy...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Final Capital:  10689.633745993953
Total Return:  6.896337459939539
Benchmark Return:  Ticker
^GSPC    21.301992
dtype: float64
Alpha:  Ticker
^GSPC   -14.405655
dtype: float64
Number of Trades:  13
Win Rate:  46.15384615384615

Testing BREAKOUT strategy
Running backtest with breakout strategy...
Final Capital:  10561.454409550417
Total Return:  5.61454409550417
Benchmark Return:  Ticker
^GSPC    21.301992
dtype: float64
Alpha:  Ticker
^GSPC   -15.687448
dtype: float64
Number of Trades:  10
Win Rate:  30.0

STRATEGY COMPARISON
                Total Return (%)                      Benchmark Return (%)  \
kronos                 -4.752485  Ticker
^GSPC    21.301992
dtype: float64   
trend_following        -2.548961  Ticker
^GSPC    21.301992
dtype: float64   
mean_reversion         16.467513  Ticker
^GSPC    21.301992
dtype: float64   
macd_crossover         14.743377  Ticker
^GSPC    21.301992
dtype: float64   
rsi_momentum            6.896337  Ticker
^GSPC    21.301992
dtype: float64   